In [1]:
import keras
import datetime
import tensorflow                       as tf
from tensorflow.keras.callbacks         import TensorBoard
from tensorflow.keras.layers            import Input, Lambda,UpSampling2D, Conv2D,Dropout,MaxPooling2D,Conv2DTranspose,concatenate,BatchNormalization, Activation
from tensorflow.keras.models            import Model
from tensorflow.keras.optimizers        import Adam,RMSprop
from keras.utils                        import plot_model
from tensorflow.keras                   import layers, models
from tensorflow.keras.losses            import mae
import sys
import os
import numpy as np
import math
import random, time
from pathlib                        import Path
from PIL                            import Image

import skimage                      as ski
from   skimage.filters              import threshold_otsu
from   skimage                      import io, color
from   skimage.color                import rgb2gray
from   skimage                      import filters
import cv2                          as cv
import matplotlib.pyplot            as plt 
import gc
import glob
from skimage                        import img_as_ubyte
from skimage                        import io
import shutil
tf.keras.backend.clear_session()

2025-05-12 20:09:48.961847: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747094988.975578   26085 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747094988.979507   26085 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747094988.989574   26085 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747094988.989588   26085 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747094988.989589   26085 computation_placer.cc:177] computation placer alr

### Working with ImageDataGenerator

In [2]:
general_directory ='/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5'

def dataset(directory,batch_size=32):
    data = keras.utils.image_dataset_from_directory(directory,
    labels=None,
    label_mode=None,
    image_size=(128, 256),
    color_mode="grayscale",
    shuffle=False,
    batch_size=batch_size)
    return data

path_1= general_directory + '/geo_masked' + '/train'
path_2= general_directory + '/geo_masked' + '/test'
path_3= general_directory + '/geo_masked' + '/valid'

path_4= general_directory + '/mag_masked' + '/train'
path_5= general_directory + '/mag_masked' + '/test'
path_6= general_directory + '/mag_masked' + '/valid'

path_7= general_directory + '/vx_masked' + '/train'
path_8= general_directory + '/vx_masked' + '/test'
path_9= general_directory + '/vx_masked' + '/valid'

path_10= general_directory + '/vy_masked' + '/train'
path_11= general_directory + '/vy_masked' + '/test'
path_12= general_directory + '/vy_masked' + '/valid'

path_13= general_directory + '/p_masked' + '/train'
path_14= general_directory + '/p_masked' + '/test'
path_15= general_directory + '/p_masked' + '/valid'
    

In [3]:
g_train = dataset(path_1)
g_test  = dataset(path_2)
g_valid = dataset(path_3)
print('******************')
v_train = dataset(path_4)
v_test  = dataset(path_5)
v_valid = dataset(path_6)
print('******************')
vx_train = dataset(path_7)
vx_test  = dataset(path_8)
vx_valid = dataset(path_9)
print('******************')
vy_train = dataset(path_10)
vy_test  = dataset(path_11)
vy_valid = dataset(path_12)
print('******************')
p_train = dataset(path_13)
p_test = dataset(path_14)
p_valid  = dataset(path_15)

Found 14000 files.
Found 2000 files.
Found 4000 files.


2025-05-12 20:09:59.122314: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-05-12 20:09:59.122332: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-05-12 20:09:59.122336: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: ppgi-System-Product-Name
2025-05-12 20:09:59.122337: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:190] hostname: ppgi-System-Product-Name
2025-05-12 20:09:59.122407: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:197] libcuda reported version is: 530.30.2
2025-05-12 20:09:59.122418: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:201] kernel rep

******************
Found 14000 files.
Found 2000 files.
Found 4000 files.
******************
Found 14000 files.
Found 2000 files.
Found 4000 files.
******************
Found 14000 files.
Found 2000 files.
Found 4000 files.
******************
Found 14000 files.
Found 2000 files.
Found 4000 files.


In [4]:
# combination of output datasets
output_train = tf.data.Dataset.zip((v_train,vx_train,vy_train,p_train))
output_test = tf.data.Dataset.zip((v_test,vx_test,vy_test,p_test))
output_valid = tf.data.Dataset.zip((v_valid,vx_valid,vy_valid,p_valid))

# combination of input datasets with output datasets
final_train = tf.data.Dataset.zip((g_train,output_train))
final_test = tf.data.Dataset.zip((g_test,output_test))
final_valid = tf.data.Dataset.zip((g_valid,output_valid))

### Standarize the data

In [5]:
# Standardizing values to be in the [0, 1] range by using tf.keras.layers.Rescaling:
normalization_layer = tf.keras.layers.Rescaling(1./255)

final_train_normalized = final_train.map(lambda x, y: (normalization_layer(x), y))
final_test_normalized = final_test.map(lambda x, y: (normalization_layer(x), y))
final_valid_normalized = final_valid.map(lambda x, y: (normalization_layer(x), y))


### Hiperparámeters

In [10]:
num_epochs = 5
patience=5  # How long to wait after last time validation loss improved
LR=0.001
# Model name
model_name="U-Net"
plot_name='U-Net_Model.png'
save_in='/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5'
# image dimensions
img_width             =  256   # 739   G:737
img_height            =  128   # 185
channel               =  1

In [11]:
number_of_filters = [16, 32, 64, 128, 256, 512]
type_padding = 'same'
f_activation = 'relu'
f_activation_last='linear'
optimizer = RMSprop(learning_rate=LR)
#number_of_filters = [64, 128, 256, 512, 1024]

### Architecture

In [12]:
def conv_block_batchnorm(filters,x):
    conv = Conv2D(filters, (3, 3), padding=type_padding)(x)
    conv= BatchNormalization()(conv)
    conv = Activation(f_activation)(conv)
    conv = Conv2D(filters, (3, 3), padding=type_padding)(conv)
    conv= BatchNormalization()(conv)
    conv = Activation(f_activation)(conv)
    return conv
    
def conv_block(filters,x):
    conv = Conv2D(filters, (3, 3), activation=f_activation, padding=type_padding)(x)
    conv = Conv2D(filters, (3, 3), activation=f_activation, padding=type_padding)(conv)
    return conv    

def encoder(x,filters):
    conv = conv_block_batchnorm(filters,x)
    downsample = MaxPooling2D((2,2))(conv)
    return conv,downsample

def decoder(x1,x2,filters,transpose=None):
    if transpose != None:
        conv_up = Conv2DTranspose(filters,(2,2),strides=(2, 2),padding=type_padding)(x1)
    else:
        conv_up = UpSampling2D((2, 2))(x1)
    print('upsampling',conv_up.shape)
    concat=concatenate([conv_up,x2],axis = 3)
    up = conv_block(filters, concat)
    return up


In [13]:
def make_model():
    
    image_input = Input((img_height, img_width, channel))
    
    conv1,down_block1 = encoder (image_input, number_of_filters[0])
    conv2,down_block2 = encoder (down_block1 , number_of_filters[1])
    conv3,down_block3 = encoder (down_block2, number_of_filters[2])
    conv4,down_block4 = encoder (down_block3, number_of_filters[3])
    conv5,down_block5 = encoder (down_block4, number_of_filters[4])
    bottleneck = conv_block(number_of_filters[5],down_block5)
    # First decoder
    up6 = decoder(bottleneck,conv5,number_of_filters[4])
    up7 = decoder(up6,conv4,number_of_filters[3])
    up8 = decoder(up7,conv3,number_of_filters[2])
    up9 = decoder(up8,conv2,number_of_filters[1])
    up10 = decoder(up9,conv1,number_of_filters[0])
    v_out = Conv2D(1, (1, 1), activation=f_activation_last,name='v_output')(up10)

    # Second decoder
    up6 = decoder(bottleneck,conv5,number_of_filters[4])
    up7 = decoder(up6,conv4,number_of_filters[3])
    up8 = decoder(up7,conv3,number_of_filters[2])
    up9 = decoder(up8,conv2,number_of_filters[1])
    up10 = decoder(up9,conv1,number_of_filters[0])
    vx_out = Conv2D(1, (1, 1), activation=f_activation_last,name='vx_output')(up10)

    # Third decoder
    up6 = decoder(bottleneck,conv5,number_of_filters[4])
    up7 = decoder(up6,conv4,number_of_filters[3])
    up8 = decoder(up7,conv3,number_of_filters[2])
    up9 = decoder(up8,conv2,number_of_filters[1])
    up10 = decoder(up9,conv1,number_of_filters[0])
    vy_out = Conv2D(1, (1, 1), activation=f_activation_last,name='vy_output')(up10)

    # Fourth decoder
    up6 = decoder(bottleneck,conv5,number_of_filters[4])
    up7 = decoder(up6,conv4,number_of_filters[3])
    up8 = decoder(up7,conv3,number_of_filters[2])
    up9 = decoder(up8,conv2,number_of_filters[1])
    up10 = decoder(up9,conv1,number_of_filters[0])
    p_out = Conv2D(1, (1, 1), activation=f_activation_last,name='p_output')(up10)

    # construct model
    model =  keras.Model(inputs=image_input, outputs=[v_out,vx_out,vy_out,p_out],name= model_name)
    
    model.summary()
    
    model.compile(optimizer=optimizer, 
              loss = {  'v_output': 'mean_squared_error' ,
                        'vx_output': 'mean_squared_error' , 
                        'vy_output': 'mean_squared_error',
                        'p_output': 'mean_squared_error'},
              
              metrics={ 'v_output': ['mae'],
                        'vx_output': ['mae'],
                        'vy_output': ['mae'],
                        'p_output': ['mae'] })
    return model             
    

In [14]:
M=make_model()

upsampling (None, 8, 16, 512)
upsampling (None, 16, 32, 256)
upsampling (None, 32, 64, 128)
upsampling (None, 64, 128, 64)
upsampling (None, 128, 256, 32)
upsampling (None, 8, 16, 512)
upsampling (None, 16, 32, 256)
upsampling (None, 32, 64, 128)
upsampling (None, 64, 128, 64)
upsampling (None, 128, 256, 32)
upsampling (None, 8, 16, 512)
upsampling (None, 16, 32, 256)
upsampling (None, 32, 64, 128)
upsampling (None, 64, 128, 64)
upsampling (None, 128, 256, 32)
upsampling (None, 8, 16, 512)
upsampling (None, 16, 32, 256)
upsampling (None, 32, 64, 128)
upsampling (None, 64, 128, 64)
upsampling (None, 128, 256, 32)


Model: "U-Net"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 256,  │        160 │ input_layer_1[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128, 256,  │         64 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 128, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 256,  │      2,320 │ activation[0][0]  │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 256,  │         64 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 128, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 128,   │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 128,   │      4,640 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 128,   │        128 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 64, 128,   │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 128,   │      9,248 │ activation_2[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 128,   │        128 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 64, 128,   │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 64,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 64,    │     18,496 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 64,    │        256 │ conv2d_4[0][0]  

 Total params: 17,297,076 (65.98 MB)

 Trainable params: 17,295,092 (65.98 MB)

 Non-trainable params: 1,984 (7.75 KB)

In [ ]:
print("Starting trainig")

history = M.fit(final_train,epochs=num_epochs,validation_data = final_valid)

Starting trainig
Epoch 1/5
 43/438 ━━━━━━━━━━━━━━━━━━━━ 59:09 9s/step - loss: 378505728.0000 - p_output_loss: 5976687.5000 - p_output_mae: 603.6033 - v_output_loss: 495816.7812 - v_output_mae: 191.1334 - vx_output_loss: 3189028.0000 - vx_output_mae: 444.6696 - vy_output_loss: 368844192.0000 - vy_output_mae: 4515.1006

In [ ]:
results = model.evaluate(final_test) 
print("Evaluation results:")
for name, value in zip(model.metrics_names, results):
    print(f"{name}: {value:.4f}")

In [ ]:
for x_test,y_true in final_test.take(1):
    y_pred = model.predict(x_test)
    for i in range(4):
        fig, axs = plt.subplots(1, 3, figsize=(12, 4))
        axs[0].imshow(x_test[0],cmap='gray')
        axs[0].set_title("Input")

        axs[1].imshow(y_true[i][0],cmap='gray')
        axs[1].set_title(f"True Output {i+1}")

        axs[2].imshow(y_pred[i][0],cmap='gray')
        axs[2].set_title(f"Pred Output {i+1}")
        plt.show()
        

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def conv_block(x, filters):
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
    return x

def encoder_block(x, filters):
    c = conv_block(x, filters)
    p = layers.MaxPooling2D((2, 2))(c)
    return c, p

def decoder_block(x, skip, filters):
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Concatenate()([x, skip])
    x = conv_block(x, filters)
    return x

def unet_with_3_decoders(input_shape=(128, 128, 1)):
    inputs = tf.keras.Input(input_shape)

    # Encoder
    c1, p1 = encoder_block(inputs, 16)
    c2, p2 = encoder_block(p1, 32)
    c3, p3 = encoder_block(p2, 64)

    # Bottleneck
    b = conv_block(p3, 128)

    # Decoder 1 (e.g., salida u)
    d1_1 = decoder_block(b, c3, 64)
    d1_2 = decoder_block(d1_1, c2, 32)
    d1_3 = decoder_block(d1_2, c1, 16)
    out_u = layers.Conv2D(1, 1, activation='linear', name='u_output')(d1_3)

    # Decoder 2 (e.g., salida v)
    d2_1 = decoder_block(b, c3, 64)
    d2_2 = decoder_block(d2_1, c2, 32)
    d2_3 = decoder_block(d2_2, c1, 16)
    out_v = layers.Conv2D(1, 1, activation='linear', name='v_output')(d2_3)

    # Decoder 3 (e.g., salida presión)
    d3_1 = decoder_block(b, c3, 64)
    d3_2 = decoder_block(d3_1, c2, 32)
    d3_3 = decoder_block(d3_2, c1, 16)
    out_p = layers.Conv2D(1, 1, activation='linear', name='p_output')(d3_3)

    # Modelo final
    model = Model(inputs, [out_u, out_v, out_p])
    return model
